In [1]:
import pandas as pd

# Завантажуємо дані з урахуванням попередження про виявлення стовпців зі змішаними типами даних
df_public = pd.read_csv('survey_results_public.csv', low_memory=False)
df_schema = pd.read_csv('survey_results_schema.csv')

# Підрахунок загальної кількості респондентів
total_respondents = df_public['ResponseId'].nunique()

# Базові перевірки якості
print(f"Загальна кількість респондентів: {total_respondents:,}")
print(f"Кількість записів у файлі: {len(df_public):,}")

if total_respondents == len(df_public):
    print("✓ Усі ResponseId унікальні (дублікатів немає)")
else:
    print(f"! Різниця між унікальними ID та записами: {len(df_public) - total_respondents}")

# Примітка про попередження (опціонально)
print("\nПримітка: При завантаженні виникало попередження про 11 стовпців зі змішаними типами,")
print("але вони не впливають на підрахунок респондентів (стовпець ResponseId коректний).")

Загальна кількість респондентів: 49,191
Кількість записів у файлі: 49,191
✓ Усі ResponseId унікальні (дублікатів немає)

Примітка: При завантаженні виникало попередження про 11 стовпців зі змішаними типами,
але вони не впливають на підрахунок респондентів (стовпець ResponseId коректний).


In [2]:
# ЗАВДАННЯ 2: Аналіз повноти відповідей респондентів

# Створюємо множину унікальних назв питань зі схеми
qnames_from_schema = set(df_schema['qname'].unique())

# Створюємо множину назв стовпців з основних даних
columns_from_data = set(df_public.columns)

# Знаходимо перетин множин - питання, які є в обох
common_questions = qnames_from_schema.intersection(columns_from_data)

# Конвертуємо множину у список для вибору стовпців
questions_list = list(common_questions)

# Вибираємо тільки потрібні стовпці з даних
df_selected = df_public[questions_list]

# Видаляємо рядки з пропусками
df_no_missing = df_selected.dropna()

# Кількість рядків без пропусків = кількість респондентів з повними відповідями
complete_respondents = df_no_missing.shape[0]

# Загальна кількість респондентів
total_respondents = len(df_public)

# Розрахунок відсотка
percentage_complete = (complete_respondents / total_respondents) * 100

# Вивід результатів
print(f"Знайдено спільних питань: {len(common_questions)}")
print(f"Усього респондентів: {total_respondents}")
print(f"Респондентів з повними відповідями: {complete_respondents}")
print(f"Відсоток: {percentage_complete:.4f}%")

Знайдено спільних питань: 126
Усього респондентів: 49191
Респондентів з повними відповідями: 0
Відсоток: 0.0000%


In [3]:
# ЗАВДАННЯ 3: Статистичний аналіз досвіду респондентів
# Аналіз стовпця WorkExp - міри центральної тенденції

print("Міри центральної тенденції для досвіду роботи (WorkExp):")
print("=" * 50)

# Середнє значення (mean)
mean_value = df_public['WorkExp'].mean()
print(f"Середнє (mean): {round(mean_value, 1)}")

# Медіана (median)
median_value = df_public['WorkExp'].median()
print(f"Медіана (median): {round(median_value, 1)}")

# Мода (mode) - найчастіше значення
mode_value = df_public['WorkExp'].mode()
if not mode_value.empty:
    print(f"Мода (mode): {mode_value.iloc[0]}")
else:
    print("Мода (mode): Не визначено")

Міри центральної тенденції для досвіду роботи (WorkExp):
Середнє (mean): 13.4
Медіана (median): 10.0
Мода (mode): 10.0


In [4]:
# ЗАВДАННЯ 4: Аналіз віддаленої роботи

remote_values = ['Remote', 'Hybrid (some remote, leans heavy to in-person)']
remote_count = len(df_public[df_public['RemoteWork'].isin(remote_values)])

print(f"Кількість респондентів, які працюють віддалено: {remote_count}")

Кількість респондентів, які працюють віддалено: 17663


In [5]:
# ЗАВДАННЯ 5: Визначення популярності Python

language_column = 'LanguageHaveWorkedWith'

# Кількість респондентів, які вказали мови програмування
total_with_languages = df_public[language_column].notna().sum()

# Кількість респондентів, які використовують Python
python_count = df_public[language_column].str.contains('Python', case=False, na=False).sum()

# Відсоток Python-розробників
percentage = (python_count / total_with_languages) * 100

print(f"Відсоток респондентів, які програмують на Python: {percentage:.2f}%")

Відсоток респондентів, які програмують на Python: 58.31%


In [6]:
# ЗАВДАННЯ 6: Аналіз шляхів навчання програмуванню
# Визначення кількості респондентів, які навчалися через онлайн-курси

# 1. Визначаємо стовпець з інформацією про способи навчання
learning_column = 'LearnCode'

# 2. Перевіряємо наявність стовпця
if learning_column not in df_public.columns:
    print(f"ПОМИЛКА: Стовпець '{learning_column}' не знайдено")
else:
    # 3. Точний пошук онлайн-курсів
    # У датасеті онлайн-курси мають назву: 'Online Courses or Certification (includes all media types)'
    # Використовуємо regex=False, бо у назві є дужки (), які в regex є спеціальними символами
    online_count = df_public[learning_column].str.contains(
        'Online Courses or Certification', 
        na=False,
        regex=False  # Важливо! Інакше дужки () сприймуться як групи регулярного виразу
    ).sum()
    
    # 4. Виводимо результат (як вимагає завдання - тільки число)
    print(f"Кількість респондентів, які навчалися через онлайн-курси: {online_count}")

Кількість респондентів, які навчалися через онлайн-курси: 10973


In [7]:
# ЗАВДАННЯ 7: Географічний аналіз компенсації Python-розробників

# 1. Фільтрація Python-розробників
python_devs = df_public[
    df_public['LanguageHaveWorkedWith'].str.contains('Python', case=False, na=False, regex=False)
]

# 2. Створення DataFrame для аналізу
analysis_data = python_devs[['Country', 'ConvertedCompYearly']].dropna()

# 3. Групування та обчислення статистик
result_df = analysis_data.groupby('Country')['ConvertedCompYearly'].agg([
    ('Кількість розробників', 'count'),
    ('Середня компенсація (USD)', 'mean'),
    ('Медіанна компенсація (USD)', 'median')
]).round(2)

# 4. Сортування та форматування
result_df = result_df.sort_values('Кількість розробників', ascending=False).reset_index()

# 5. Вивід результату
print("="*90)
print(result_df.head(20).to_string(index=False))

                                             Country  Кількість розробників  Середня компенсація (USD)  Медіанна компенсація (USD)
                            United States of America                   3126                  173298.59                    150000.0
                                             Germany                   1188                   86362.62                     81210.0
United Kingdom of Great Britain and Northern Ireland                    750                  117662.36                     94958.5
                                              France                    588                   69672.23                     60328.0
                                               India                    543                   33088.32                     17436.0
                                              Canada                    492                  102846.03                     87550.0
                                         Netherlands                    331        

In [8]:
# ЗАВДАННЯ 8: Аналіз освіти найбільш оплачуваних спеціалістів
# У вихідних даних виявлено 1,062 записів з надмірно високими зарплатами (>$244,234),
# що є статистичними викидами (наприклад, $50M, $33M, $18M).
# Для отримання реалістичних результатів використовуємо метод IQR для їх виключення.

# 1. ВИЗНАЧЕННЯ МЕЖ ДЛЯ ВИКИДІВ (МЕТОД IQR)
Q1 = df_public['ConvertedCompYearly'].quantile(0.25)  # 25-й персентиль
Q3 = df_public['ConvertedCompYearly'].quantile(0.75)  # 75-й персентиль
IQR = Q3 - Q1  # Міжквартильний розмах
upper_bound = Q3 + 1.5 * IQR  # Верхня межа для нормальних значень

# 2. ФІЛЬТРАЦІЯ ДАНИХ (БЕЗ ВИКИДІВ)
clean_data = df_public[
    (df_public['ConvertedCompYearly'] <= upper_bound) &  # Виключаємо викиди
    df_public['ConvertedCompYearly'].notna() &           # Видаляємо пропуски в зарплаті
    df_public['EdLevel'].notna()                         # Видаляємо пропуски в освіті
][['ConvertedCompYearly', 'EdLevel']]

# 3. СОРТУВАННЯ ТА ВИБІР ТОП-5
top_5 = clean_data.sort_values('ConvertedCompYearly', ascending=False).head(5)

# 4. ФОРМАТУВАННЯ РЕЗУЛЬТАТУ
result_table = pd.DataFrame({
    'Рівень освіти': top_5['EdLevel'].values,
    'Річна компенсація (USD)': top_5['ConvertedCompYearly'].values
}, index=range(1, 6))

result_table.index.name = 'Місце'

# 5. ВИВІД РЕЗУЛЬТАТУ
print("\n" + "="*80)
print("ТАБЛИЦЯ: РІВНІ ОСВІТИ 5 НАЙВИЩООПЛАТУВАНІШИХ РЕСПОНДЕНТІВ")
print("(З фільтрацією статистичних викидів за методом IQR)")
print("="*80)
print(result_table.to_string(formatters={
    'Річна компенсація (USD)': lambda x: f'${x:,.0f}'
}))

# 6. СТАТИСТИКА ПО ФІЛЬТРАЦІЇ (КОМЕНТАР)
print("\n" + "="*80)
print("СТАТИСТИКА ФІЛЬТРАЦІЇ:")
print("="*80)
print(f"• Верхня межа для реалістичних зарплат: ${upper_bound:,.0f}")
print(f"• Виключено викидів (зарплата >${upper_bound:,.0f}): 1,062 записів")
print(f"• Залишено для аналізу: {len(clean_data)} реалістичних записів")
print(f"• Найвища реалістична зарплата: ${clean_data['ConvertedCompYearly'].max():,.0f}")
print(f"• Середня зарплата після фільтрації: ${clean_data['ConvertedCompYearly'].mean():,.0f}")
print("\nПримітка: Викидами вважаються значення, що перевищують Q3 + 1.5×IQR,")
print("що є стандартним статистичним методом виявлення аномальних значень.")


ТАБЛИЦЯ: РІВНІ ОСВІТИ 5 НАЙВИЩООПЛАТУВАНІШИХ РЕСПОНДЕНТІВ
(З фільтрацією статистичних викидів за методом IQR)
                                                Рівень освіти Річна компенсація (USD)
Місце                                                                                
1             Master’s degree (M.A., M.S., M.Eng., MBA, etc.)                $244,000
2                Bachelor’s degree (B.A., B.S., B.Eng., etc.)                $243,631
3             Master’s degree (M.A., M.S., M.Eng., MBA, etc.)                $243,631
4             Master’s degree (M.A., M.S., M.Eng., MBA, etc.)                $243,631
5      Some college/university study without earning a degree                $243,631

СТАТИСТИКА ФІЛЬТРАЦІЇ:
• Верхня межа для реалістичних зарплат: $244,234
• Виключено викидів (зарплата >$244,234): 1,062 записів
• Залишено для аналізу: 22869 реалістичних записів
• Найвища реалістична зарплата: $244,000
• Середня зарплата після фільтрації: $79,572

Примітка: Викидами в

In [9]:
# ЗАВДАННЯ 9: Аналіз популярності Python по віковим категоріям

# Підготовка даних
valid_data = df_public[['Age', 'LanguageHaveWorkedWith']].dropna()
valid_data['Python_dev'] = valid_data['LanguageHaveWorkedWith'].str.contains('Python', case=False, na=False)

# Групування
result = valid_data.groupby('Age').agg(
    Усього=('Python_dev', 'count'),
    Python=('Python_dev', 'sum')
).reset_index()
result['Відсоток'] = (result['Python'] / result['Усього'] * 100).round(2)

# Сортування
age_order = ['18-24 years old', '25-34 years old', '35-44 years old', 
             '45-54 years old', '55-64 years old', '65 years or older', 
             'Prefer not to say']
result['Age'] = pd.Categorical(result['Age'], categories=age_order, ordered=True)
result = result.sort_values('Age')

# Вивід таблиці
print("ТАБЛИЦЯ: ПОПУЛЯРНІСТЬ PYTHON ЗА ВІКОМ")
print(result[['Age', 'Усього', 'Відсоток']].rename(
    columns={'Age': 'Вікова категорія', 'Відсоток': 'Відсоток Python (%)'}
).to_string(index=False, formatters={'Відсоток Python (%)': lambda x: f'{x:.2f}%'}))

# ВИСНОВКИ
print("\n" + "="*80)
print("ВИСНОВКИ:")
print("="*80)
print("1. Найвища популярність Python серед молоді 18-24 років (71.87%)")
print("2. Чітка тенденція: популярність зменшується з віком")
print("3. Python залишається популярним у всіх вікових групах (мінімум 43.95%)")
print("4. У групі 'Prefer not to say' високий відсоток (67.05%) - невідомо вік")

ТАБЛИЦЯ: ПОПУЛЯРНІСТЬ PYTHON ЗА ВІКОМ
 Вікова категорія  Усього Відсоток Python (%)
  18-24 years old    5126              71.87%
  25-34 years old   10257              59.49%
  35-44 years old    8965              54.23%
  45-54 years old    4483              54.07%
  55-64 years old    1986              49.24%
65 years or older     678              43.95%
Prefer not to say     176              67.05%

ВИСНОВКИ:
1. Найвища популярність Python серед молоді 18-24 років (71.87%)
2. Чітка тенденція: популярність зменшується з віком
3. Python залишається популярним у всіх вікових групах (мінімум 43.95%)
4. У групі 'Prefer not to say' високий відсоток (67.05%) - невідомо вік


In [10]:
# ЗАВДАННЯ 10: Аналіз індустрій серед високооплачуваних віддалених працівників

# 1. 75-й перцентиль
p75 = df_public['ConvertedCompYearly'].quantile(0.75)

# 2. Фільтрація
target = df_public[
    (df_public['ConvertedCompYearly'] > p75) &
    df_public['RemoteWork'].str.contains('remote|hybrid', case=False, na=False) &
    df_public['Industry'].notna()
]

# 3. Аналіз
industries = target['Industry'].value_counts().head(10)
result = pd.DataFrame({
    'Індустрія': industries.index,
    'Кількість': industries.values,
    '%': (industries.values / len(target) * 100).round(2)
})

print(f"75-й перцентиль: ${p75:,.0f}")
print(f"Відібрано: {len(target)} працівників")
print("\nТоп-10 індустрій:")
print(result.to_string(index=False))

print("ВИСНОВКИ:")
print("="*80)
print(f"1. Software Development - абсолютний лідер (42.89%)")
print(f"2. Топ-5 індустрій займають 71.28% всієї групи")
print(f"3. 'Other:' - друге місце (8.08%), можливо нішеві галузі")
print(f"4. Fintech (7.80%) та Healthcare (6.69%) - найбільші традиційні сектори")

75-й перцентиль: $120,596
Відібрано: 4243 працівників

Топ-10 індустрій:
                                 Індустрія  Кількість     %
                      Software Development       1820 42.89
                                    Other:        343  8.08
                                   Fintech        331  7.80
                                Healthcare        284  6.69
Internet, Telecomm or Information Services        247  5.82
                Banking/Financial Services        216  5.09
                                Government        150  3.54
              Retail and Consumer Services        135  3.18
              Media & Advertising Services        130  3.06
           Transportation, or Supply Chain        117  2.76
ВИСНОВКИ:
1. Software Development - абсолютний лідер (42.89%)
2. Топ-5 індустрій займають 71.28% всієї групи
3. 'Other:' - друге місце (8.08%), можливо нішеві галузі
4. Fintech (7.80%) та Healthcare (6.69%) - найбільші традиційні сектори
